In [ ]:
import requests
import re
import time

import pandas as pd
import lxml
from bs4 import BeautifulSoup
from unicodedata import normalize
from geopy.geocoders import Nominatim
from haversine import haversine
import matplotlib.pyplot as plt
import numpy as np
from openpyxl import load_workbook
import xlsxwriter

pd.options.display.max_rows = 6000

In [ ]:
URL = 'https://www.searchfunder.com'

session = requests.session()

front = session.get(URL)
csrf_token = re.findall(r'<input type="hidden" name="_token" value="(.*)"', front.text)[0]

cookies = session.cookies

payload = {
    'email': 'mhill@nextgengp.com',
    'password': 'ETA1234',
    '_token': csrf_token,
}

r = requests.post(URL + '/auth/login', data=payload, cookies=cookies)


In [ ]:
searchfunds_HTML = requests.get(URL + '/searchfund/allsearchfunds', cookies=r.cookies)

In [ ]:
datasearchfund_id = re.findall('data-searchfund_id="[0-9]+"',searchfunds_HTML.text)
searchfund_data_index = list(map(int,re.findall('[0-9]+',str(datasearchfund_id))))

In [ ]:
len(searchfund_data_index)

In [ ]:
table = pd.read_html(searchfunds_HTML.text)
searchfunds = table[0][table[0]['type'].notna()]
searchfunds = searchfunds.reset_index()
searchfunds

In [ ]:
table = pd.read_html(searchfunds_HTML.text)
searchfunds = table[0][table[0]['type'].notna()]
searchfunds = searchfunds.reset_index()
searchfunds.index = searchfund_data_index
cols = [0,1]
searchfunds.drop(searchfunds.columns[cols], axis=1,inplace=True)
#searchfunds.to_csv("searchfund_raw2.csv")

In [ ]:
#enrichment = []

In [ ]:
#enrichment

In [ ]:
#enrichment.loc[enrichment['searchfund_index'] == 10]

In [ ]:
type(searchfund_data_index[-1])

In [ ]:
enrichment = []
for index in searchfund_data_index:
    print(index)
    data = requests.get(URL + '/searchfund/ajaxsearchfundrow/'+str(index), cookies=r.cookies)
    time.sleep(0.05)
    soup = BeautifulSoup(data.content, 'html.parser')
    
    ## Get company info
    company_rows=soup.find_all(class_="col-md-3")
    try:
        searchWebsite = re.split(r'\t+',company_rows[0].text)[1].strip('\n')
    except:
        searchWebsite = ""
    try:
        companyWebsite= re.split(r'\t+',company_rows[1].text)[1].strip('\n')
    except:
        companyWebsite=""
    try:
        industry = re.split(r'\t+',company_rows[2].text)[1].strip('\n')
    except:
        industry=""
    try:
        location = re.split(r'\t+',company_rows[3].text)[1].strip('\n')
    except:
        location=""

    
    
    ## Get Searcher info
    searchers=[]
    try:
        searcher_rows=soup.find_all(class_="col-md-12")[0]
        for searcher_row in searcher_rows:
            try:
                searchers.append(re.split(r'\t+',searcher_row.text)[-1].strip('\n'))
            except:
                searchers.append("")
        searchers = list(filter(None, searchers))
    except:
        searchers


    ## Get Investor info
    investors=[]
    try:
        investor_rows=soup.find_all(class_="col-md-12")[2]
        for investor_row in investor_rows:
            try:
                investors.append(re.split(r'\t+',investor_row.text)[-1].strip('\n'))
            except:
                investors.append("")
        investors = list(filter(None, investors))
    except:
        investors

    ## Get Associate info
    associates=[]
    try:
        associate_rows=soup.find_all(class_="col-md-12")[4]
        for associate_row in associate_rows:
            try:
                associates.append(re.split(r'\t+',associate_row.text)[-1].strip('\n'))
            except:
                associates.append("")
        associates = list(filter(None, associates))
    except:
        associates
    
    enrichmentData = {'searchfund_index': index, 'searchWebsite': searchWebsite,'companyWebsite': companyWebsite,'industry':industry,'location':location,'searchers':searchers,'investors':investors,'associates':associates}
    #print(enrichmentData)
    enrichment.append(enrichmentData)
    
    

In [ ]:
enrichmentDF = pd.DataFrame(enrichment)


In [ ]:
data = requests.get(URL + '/searchfund/ajaxsearchfundrow/6', cookies=r.cookies)

In [ ]:
soup = BeautifulSoup(data.content, 'html.parser')

In [ ]:
## Get company info
company_rows=soup.find_all(class_="col-md-3")
try:
    searchWebsite = re.split(r'\t+',company_rows[0].text)[1].strip('\n')
except:
    searchWebsite = ""
try:
    companyWebsite= re.split(r'\t+',company_rows[1].text)[1].strip('\n')
except:
    companyWebsite=""
try:
    industry = re.split(r'\t+',company_rows[2].text)[1].strip('\n')
except:
    industry=""
try:
    location = re.split(r'\t+',company_rows[3].text)[1].strip('\n')
except:
    location=""
    
    
## Get Searcher info
searchers=[]
try:
    searcher_rows=soup.find_all(class_="col-md-12")[0]
    for searcher_row in searcher_rows:
        try:
            searchers.append(re.split(r'\t+',searcher_row.text)[-1].strip('\n'))
        except:
            searchers.append("")
    searchers = list(filter(None, searchers))
    print(searchers)
except:
    searchers
    
    
## Get Investor info
investors=[]
try:
    investor_rows=soup.find_all(class_="col-md-12")[2]
    for investor_row in investor_rows:
        try:
            investors.append(re.split(r'\t+',investor_row.text)[-1].strip('\n'))
            index= index +1
        except:
            investors.append("")
    investors = list(filter(None, investors))
    print(investors)
except:
    investors
    
## Get Associate info
associates=[]
try:
    associate_rows=soup.find_all(class_="col-md-12")[4]
    for associate_row in associate_rows:
        try:
            associates.append(re.split(r'\t+',associate_row.text)[-1].strip('\n'))
        except:
            associates.append("")
    associates = list(filter(None, associates))
    print(associates)
except:
    associates

In [ ]:
enrichmentDF


In [ ]:
#searchfunds = pd.merge([searchfunds, enrichment], how='right',on='searchfund_index')

merged = pd.merge(searchfunds, enrichmentDF, left_index=True, right_on='searchfund_index')
merged

In [ ]:
merged['name'] = merged['name'].str.split("|", n = 1, expand = True)[0]
merged['formed'] = merged['formed'].str.split(" ", n = 1, expand = True)[0]
merged['acquired'] = merged['acquired'].str.split(" ", n = 1, expand = True)[0]

In [ ]:
merged

In [ ]:
#pd.to_datetime('formed', errors = 'coerce')

In [ ]:
merged

In [ ]:
#convert to datetime
merged['formed'] = pd.to_datetime(merged['formed'].astype(str), format = '%Y%m%d', errors = 'coerce')
merged['acquired'] = pd.to_datetime(merged['acquired'].astype(str), format = '%Y%m%d', errors = 'coerce')

In [ ]:
merged

In [ ]:
# find time it took between starting a search and making an aquisition/ending, non-calculable values are listed as NaN
count = 0
searchtime = []
for val in merged['acquired']:
    if (merged['acquired'][count] - merged['formed'][count]).days < 0:
        searchtime.append('NaN')
    else:
        searchtime.append((merged['acquired'][count] - merged['formed'][count]).days)
    count += 1
merged["Search Length"] = searchtime

In [ ]:
merged

In [ ]:
#convert from list to string
##merged['investors'] = merged['investors'].apply(lambda x: x[1:-1])
#merged['associates'] = merged['associates'].apply(lambda x: x[1:-1])

In [ ]:
merged["investors"] = merged["investors"].astype(str).apply(lambda x: x[1:-1])
merged["searchers"] = merged["searchers"].astype(str).apply(lambda x: x[1:-1])
merged["associates"] = merged["associates"].astype(str).apply(lambda x: x[1:-1])
#merged.rename(columns={"location_x": "Search Location", "location_y": "Acquistion Location", "searchfund_index": "Searchfund Index"})
#mformat = mformat.replace(u"\xa0", u" ")
#mformat["searchers"][0] = mformat['searchers'][0].replace(u'\xa0', " ").encode('utf-8')

In [ ]:
#removes all ascii characters from searchers column and also removes \\xa0 delimiter
count = 0
for val in merged["searchers"]:
    merged["searchers"][count] = merged['searchers'][count].encode("ascii", 'ignore')
    asciiremove = merged["searchers"][count].decode()
    asciiremove = asciiremove.replace("\\xa0", " ")
    merged["searchers"][count] = asciiremove
    count += 1

In [ ]:
#removes all ascii characters from investors column and also removes \\xa0 delimiter
count = 0
for val in merged["investors"]:
    merged["investors"][count] = merged['investors'][count].encode("ascii", 'ignore')
    asciiremove = merged["investors"][count].decode()
    asciiremove = asciiremove.replace("\\xa0", " ")
    merged["investors"][count] = asciiremove
    count += 1

In [ ]:
#removes all ascii characters form the associates column and also removes \\xa0 delimiter
count = 0
for val in merged["associates"]:
    merged["associates"][count] = merged['associates'][count].encode("ascii", 'ignore')
    asciiremove = merged["associates"][count].decode()
    asciiremove = asciiremove.replace("\\xa0", " ")
    merged["associates"][count] = asciiremove
    count += 1

In [ ]:
geolocator = Nominatim(user_agent = 'myapplication')
count = 0
location_lat = []
location_lon = []
for val in  merged["location_x"]:
    location = geolocator.geocode(val,timeout=10)
    try:
        location_lat.append(location.raw['lat']) 
        location_lon.append(location.raw['lon'])
    except:
        location_lat.append('nan') 
        location_lon.append('nan')

In [ ]:
geolocator = Nominatim(user_agent = 'myapplication')
count = 0
acq_lat = []
acq_lon = []
for val in  merged["location_y"]:
    location = geolocator.geocode(val,timeout=10)
    try:
        acq_lat.append(location.raw['lat']) 
        acq_lon.append(location.raw['lon'])
    except:
        acq_lat.append('nan') 
        acq_lon.append('nan')

In [ ]:
merged["search long"] = location_lon
merged["search lat"] = location_lat
merged["acquisition lat"] = acq_lat
merged["acquisition long"] = acq_lon
merged = merged.replace('nan', '0')
merged['search lat'] =pd.to_numeric(merged['search lat'])
merged['search long'] =pd.to_numeric(merged['search long'])
merged['acquisition lat'] =pd.to_numeric(merged['acquisition lat'])
merged['acquisition long'] =pd.to_numeric(merged['acquisition long'])
merged['search lat'] = merged['search lat'].replace(0, "NaN")
merged['search long'] = merged['search long'].replace(0, "NaN")
merged['acquisition lat'] = merged['acquisition lat'].replace(0, "NaN")
merged['acquisition long'] = merged['acquisition long'].replace(0, "NaN")

In [ ]:
distance = []
count = 0
for val in merged['search lat']:
    try:
        distance.append(haversine((merged['search lat'][count], merged['search long'][count]), (merged['acquisition lat'][count], merged['acquisition long'][count]), unit = 'mi'))
        
    except:
        distance.append('NaN')
    count += 1
merged["Miles Between Search and Acquisiton"] = distance

In [ ]:
merged

In [ ]:
merged = merged.rename(columns={"location_x": "Search Location", "location_y": "Acquistion Location", "searchfund_index": "Searchfund Index"})

In [ ]:
merged['industry'] = merged['industry'].fillna('', inplace=True)
merged.head()

In [ ]:
####### last cell of working on merged data

In [ ]:
merged.to_csv("searchfund_allV7.csv")

In [ ]:
copy_df = pd.read_csv("searchfund_allV6.csv")

In [ ]:
copy_df

In [ ]:
#creates a pie chart of the searchfund industry interest, ignoring NaN values
industry_count = copy_df.groupby('industry').size()
size = str(industry_count.sum())
industries = industry_count.index
percent = 100 *industry_count/industry_count.sum()
patches, texts = plt.pie(industry_count, startangle = 90, radius = 1.2)
labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(industries, percent)]
sort_legend = False
if sort_legend:
    patches, labels, dummy =  zip(*sorted(zip(patches, labels, industry_count),key=lambda x: x[2], reverse=True))
plt.legend(patches, labels, loc='upper right', bbox_to_anchor=(-0.1, 1.),
           fontsize=8)

plt.title('Search Fund Industry Interest By Percentage (Sample Size: ' + size + ")")

In [ ]:
#find the success rate/failure rate/ still searching rate of each year listed
stages = copy_df['stage'].unique()

In [ ]:
count = 0
formed_years = []
for val in copy_df['formed']:
    try:
        formed_years.append(copy_df['formed'][count][:4])
    except:
        formed_years.append(None)
    count += 1
copy_df['Year Formed'] = formed_years

In [ ]:
#grouping the total searchers by year the founded, and what stage they made it to.... calculating percentage success rate
total_searchers = copy_df.groupby(["Year Formed"]).size()
stage_year = copy_df.groupby(['Year Formed', 'stage']).size()
stage_year = stage_year.unstack(level = 1)
stage_year.drop(labels = '2040', axis = 0, inplace = True)
stage_year["Total"] = total_searchers
stage_year[['Acquired','Explore', 'Inactive', 'Raise', 'Search']] = stage_year[['Acquired','Explore', 'Inactive', 'Raise', 'Search']].div(stage_year.Total, axis=0)
stage_year = stage_year.fillna(0)

In [ ]:
#outputting the stage_year dataframe to an existing excel file and sheet
book = load_workbook("output.xlsx")
writer = pd.ExcelWriter("output.xlsx", engine='openpyxl') 
writer.book = book

writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

stage_year.to_excel(writer, "By Vintage")

writer.save()

In [ ]:
workbook = xlsxwriter.Workbook('output.xlsx')
worksheet = workbook.add_worksheet('By Vintage') # Sheet1
worksheet = workbook.add_worksheet('WorkLog') # WorkLog

In [ ]:
#creating Bar plot visualizing the breakdown which stage funds are in 
stage_bar = stage_year[['Acquired', 'Explore', "Inactive", 'Raise', 'Search']].plot.bar(stacked = True)
plt.title("Stage Breakdown of Search Fund By Year")
plt.legend(loc='upper right', bbox_to_anchor=(-0.1, 1.),
           fontsize=8)
plt.savefig('Stage Barchart.png', bbox_inches = 'tight')

In [ ]:
##### Below Cells Are For Analyzing Investors on Searchfunder ####

In [ ]:
#creates a list of all investors from searchfunder
investor_list = []
for val in copy_df['investors']:
    try:
        investors = val
        investors = investors.split(',')
        investor_list.extend(investors)
    except:
        investor_list.append("NaN")

In [ ]:
#returns the list of a row index for an investor, allows you to find which fund they invested by indexing the row of the searchfunder dataframe
def get_investor(investor):
    investor_index = copy_df['investors'].str.find(investor)
    count = 0
    indexes = []
    for val in investor_index:
        if investor_index[count] >= 0:
            indexes.append(count)
        count += 1
    return indexes

In [ ]:
#creating a dictionary that contains an investors name as the key and the index of all of their investments as the value
investors_index = []
for investor in investor_list:
    investors_index.append(get_investor(investor))
investors_dict = {investor_list[i]: investors_index[i] for i in range(len(investor_list))}
investors_dict.pop("NaN", None)

In [ ]:
#creates a dictionary with the key as the investor and the companies/industies he invested in as the values
investor_businesses = []
investor_industries = []
business_dict = {key: None for key in investor_list}
for key in investors_dict:
    business_list = []
    industry_list = []
    for val in investors_dict[key]:
        business_list.append(copy_df['company'][val])
        industry_list.append(copy_df['industry'][val])
        business_dict[key] = {"Companies": business_list, "Industries": industry_list}
    investor_businesses.append(business_list)
    investor_industries.append(industry_list)
#print(investor_businesses)
#print(investor_industries)

In [ ]:
#using the investors index, creating a list that counts which stage an investor has reached in each of their investment
investor_success = []
for key in investors_dict:
    stage_list = []
    for val in investors_dict[key]:
        stage_list.append(copy_df['stage'][val])
        investors_dict[key] = stage_list
    investor_success.append(stage_list)
    
#counting the number of acquisitions an investor has made out of their total investments, then calculating their success rate
#appending this new information to the investors dictionary then formatting to a dataframe and exporting to an excel file

for key in investors_dict:
    acquired = 0
    explore = 0
    inactive = 0
    raising = 0
    search = 0
    total = len(investors_dict[key])
    
    #counting the number of each stage that exists
    for val in investors_dict[key]:
        if val == 'Acquired':
            acquired += 1
        elif val == 'Explore':
            explore += 1
        elif val == 'Inactive':
            inactive += 1
        elif val == 'Raise':
            raising += 1
        elif val == "Search":
            search += 1
    try:
        investors_dict[key] = {"Acquisitions": acquired, "Explore": explore, "Inactive": inactive, 'Raise': raising, "Search": search, "Investments": total, "Acquisition Rate": acquired/total, "Failure Rate": inactive/total, "Still Searching": (explore + raising + search)/total}
    except:
        investors_dict[key] = {"Acquisitions": acquired, "Explore": explore, "Inactive": inactive, 'Raise': raising, "Search": search, "Investments": total, "Acquisition Rate": 0, "Failure Rate": inactive/total, "Still Searching": (explore + raising + search)/total}
    

#exporting success dataframe to excel        
success_df = pd.DataFrame.from_dict(investors_dict)
success_df = success_df.T
success_df.to_csv("Investor Success Rate.CSV")

In [ ]:
#get a DF of investors with more than 30 investments
lg_investors = success_df[success_df['Acquisitions'].gt(30)].index
lg_dict = {key: None for key in lg_investors}

for val in lg_investors:
    lg_dict[val] = success_df.at[val, "Acquisition Rate"]
lg_df = pd.DataFrame.from_dict(lg_dict, orient = 'index')

In [ ]:
lg_df.describe()

In [ ]:
success_df

In [ ]:
copy_df

In [ ]:
states = ("Alabama (AL) Alaska (AK) Arizona (AZ) Arkansas (AR) California (CA) Colorado (CO) Connecticut (CT) Delaware (DE) District of Columbia (DC) Florida (FL) Georgia (GA) Hawaii (HI) Idaho (ID) Illinois (IL) Indiana (IN) Iowa (IA) Kansas (KS) Kentucky (KY) Louisiana (LA) Maine (ME) Maryland (MD) Massachusetts (MA) Michigan (MI) Minnesota (MN) Mississippi (MS) Missouri (MO) Montana (MT) Nebraska (NE) Nevada (NV) New Hampshire (NH) New Jersey (NJ) New Mexico (NM) New York (NY) North Carolina (NC) North Dakota (ND) Ohio (OH) Oklahoma (OK) Oregon (OR) Pennsylvania (PA) Rhode Island (RI) South Carolina (SC) South Dakota (SD) Tennessee (TN) Texas (TX) Utah (UT) Vermont (VT) Virginia (VA) Washington (WA) West Virginia (WV) Wisconsin (WI) Wyoming (WY)")
states = states.replace(")", "")
states = states.replace("(", "")
states = states.split(" ")
#geolocator = Nominatim(user_agent = "myapplicaiton")
search_state = []
for val in copy_df['Search Location']:
    try:
        for state in states:
            if any(state in s for s in display_name):
                search_state.append(state)
                print(search_state)
                break
            else:
                search_state.append("Not in US")
    except:
        search_state.append("NaN")

print(search_state)

In [ ]:
for val in copy_df["Search Location"]:
    print(val)

In [ ]:
states = ("Alabama (AL) Alaska (AK) Arizona (AZ) Arkansas (AR) California (CA) Colorado (CO) Connecticut (CT) Delaware (DE) District of Columbia (DC) Florida (FL) Georgia (GA) Hawaii (HI) Idaho (ID) Illinois (IL) Indiana (IN) Iowa (IA) Kansas (KS) Kentucky (KY) Louisiana (LA) Maine (ME) Maryland (MD) Massachusetts (MA) Michigan (MI) Minnesota (MN) Mississippi (MS) Missouri (MO) Montana (MT) Nebraska (NE) Nevada (NV) New Hampshire (NH) New Jersey (NJ) New Mexico (NM) New York (NY) North Carolina (NC) North Dakota (ND) Ohio (OH) Oklahoma (OK) Oregon (OR) Pennsylvania (PA) Rhode Island (RI) South Carolina (SC) South Dakota (SD) Tennessee (TN) Texas (TX) Utah (UT) Vermont (VT) Virginia (VA) Washington (WA) West Virginia (WV) Wisconsin (WI) Wyoming (WY)")
states = states.replace(")", "")
states = states.replace("(", "")
states = states.split(" ")
geolocator = Nominatim(user_agent = "myapplicaiton")
search_state = []

location = copy_df['Search Location'][20]
display_name = location.split(" ")
print(location)
print(display_name)
for state in states:
    if any(state in s for s in display_name):
        search_state.append(state)
        print(search_state)
        break

In [ ]:

states